# Layout Recognition

In [1]:
import os
import keras
import pandas
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm

%matplotlib inline

from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import load_model


Using TensorFlow backend.
EndFontMetrics quoteleft 15 150575000 407 700 ;754 ;;;on. All rights reserved. GE Inspira is a trademark of the General Electric Company.


In [2]:
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="float32" )
    return data

data_dir = 'layout_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'


In [7]:
img_width, img_height = 224, 224
batch_size = 3
epochs = 10

train_datagen = ImageDataGenerator(
    rescale = 1./224
    )

test_datagen = ImageDataGenerator(
    rescale = 1./224
    )

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size, 
    class_mode = "categorical")

validation_generator = test_datagen.flow_from_directory(
    valid_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical")

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = (img_height, img_width),
    class_mode = "categorical")

Found 23 images belonging to 2 classes.
Found 9 images belonging to 2 classes.
Found 8 images belonging to 2 classes.


In [8]:
vgg_model = applications.VGG19(weights = "imagenet", 
                           include_top=False, 
                           input_shape = (img_width, img_height, 3))
for layer in vgg_model.layers[:5]:
    layer.trainable = False
x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
predictions = Dense(2, activation='softmax')(x)

model_final = Model(inputs=vgg_model.input, outputs=predictions)
for layer in vgg_model.layers:
    layer.trainable = False
model_final.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [13]:
model_final.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.samples/train_generator.batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples/batch_size)

Epoch 1/10
7/7 [==============================] - 30s - loss: 1.0967 - val_loss: 0.9325
Epoch 2/10
7/7 [==============================] - 27s - loss: 0.5691 - val_loss: 0.5195
Epoch 3/10
7/7 [==============================] - 27s - loss: 0.5281 - val_loss: 0.3604
Epoch 4/10
7/7 [==============================] - 27s - loss: 0.3833 - val_loss: 0.3382
Epoch 5/10
7/7 [==============================] - 27s - loss: 0.2794 - val_loss: 0.2291
Epoch 6/10
7/7 [==============================] - 28s - loss: 0.2094 - val_loss: 0.1488
Epoch 7/10
7/7 [==============================] - 28s - loss: 0.1415 - val_loss: 0.1869
Epoch 8/10
7/7 [==============================] - 27s - loss: 0.1740 - val_loss: 0.0881
Epoch 9/10
7/7 [==============================] - 28s - loss: 0.0628 - val_loss: 0.0636
Epoch 10/10
7/7 [==============================] - 28s - loss: 0.0914 - val_loss: 0.0643


In [15]:
checkpoint = ModelCheckpoint("layout_vgg19_wt.h5", 
                             monitor='val_acc', verbose=2, 
                             save_best_only=True, save_weights_only=False, 
                             mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', 
                      min_delta=0, patience=3, 
                      verbose=2, mode='auto')
layer_num = len(model_final.layers)
for layer in model_final.layers[:21]:
    layer.trainable = False
for layer in model_final.layers[21:]:
    layer.trainable = True
model_final.compile(optimizer=optimizers.Adam(lr=0.0001), 
              loss='categorical_crossentropy', metrics=['accuracy'])
model_final.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples/train_generator.batch_size,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = validation_generator.samples/batch_size,
    callbacks = [checkpoint, early])

Epoch 1/10
7/7 [==============================] - 30s - loss: 0.0560 - acc: 1.0000 - val_loss: 0.0444 - val_acc: 1.0000
Epoch 2/10
7/7 [==============================] - 28s - loss: 0.0391 - acc: 1.0000 - val_loss: 0.0412 - val_acc: 1.0000
Epoch 3/10
7/7 [==============================] - 27s - loss: 0.0517 - acc: 1.0000 - val_loss: 0.0389 - val_acc: 1.0000
Epoch 4/10
7/7 [==============================] - 28s - loss: 0.0352 - acc: 1.0000 - val_loss: 0.0363 - val_acc: 1.0000
Epoch 5/10
7/7 [==============================] - 27s - loss: 0.0390 - acc: 1.0000 - val_loss: 0.0344 - val_acc: 1.0000
Epoch 00004: early stopping


In [16]:
model_final.save('layout_vgg19_model.h5')